## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Algoritmo Genético para asignación de quirófanos

#### Autor: Jesús García Armario

In [1]:
# Imports necesarios
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import sys
sys.path.append('../')
from Heuristicas.Utils import Quirofano, ActoQuirurgico
import random
import math
import matplotlib.pyplot as plt

In [2]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]
# Creamos un set de actos quirúrgicos
actos_pendientes = list()
i = 0
for elemento in df.itertuples():
    actos_pendientes.append(ActoQuirurgico(i, elemento[3], elemento[1], elemento[2]))
    i += 1

In [3]:
# Definimos un cromosoma como una lista de tamaño N
# donde N es el número de slots temporales disponibles
# El ID de la operación ocupará la posición del slot temporal o 0 si no se ha asignado
# El cromosoma se codifica como una lista de enteros
# Con separadores representados por caracteres especiales, siendo 'A' el separador de quirófanos
# y 'B' el separador de días
def cromosomaAleatorio(tiempos, quirofanos, dias, actos_pendientes):
    actos_pendientes = actos_pendientes.copy()
    cromosoma = []
    # Generamos un cromosoma aleatorio
    for i in range(dias):
        for j in range(quirofanos):
            # Seleccionamos un numero de intervenciones aleatorio para el slot
            num_intervenciones = random.randint(1, tiempos)
            # Seleccionamos las intervenciones aleatorias
            intervenciones = random.sample(actos_pendientes, num_intervenciones)
            # Las añadimos al cromosoma
            for intervencion in intervenciones:
                cromosoma.append(intervencion.getId())
                actos_pendientes.remove(intervencion)
            # Añadimos un separador de quirófanos
            cromosoma.append('A')
        # Añadimos un separador de días
        cromosoma.append('B')
    return cromosoma

In [4]:
print(cromosomaAleatorio(5, 2, 2, actos_pendientes))

[88, 95, 'A', 55, 21, 11, 'A', 'B', 57, 24, 0, 'A', 39, 16, 35, 'A', 'B']


In [5]:
# Definimos al individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("cromosoma", cromosomaAleatorio, 16, 3, 5, actos_pendientes)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.cromosoma)
toolbox.register("poblacion", tools.initRepeat, list, toolbox.individual)

In [5]:
# Definimos la función de fitness
def evaluar(individual):
    # Calculamos el fitness como la suma de las ponderaciones de los actos quirúrgicos
    # Dividida entre el número de pacientes pendientes de operar
    fitness = 0
    # Calculamos el fitness
    for i in range(len(individual)):
        if individual[i] != 'A' and individual[i] != 'B':
            fitness += actos_pendientes[individual[i]].getPrioridad()
    # Dividimos entre los slots temporales vacíos
    pendientes_operar = len(actos_pendientes)
    return fitness,

In [6]:
def cruce(ind1, ind2):
    # Obtenemos las posiciónes de los separadores A y B 
    posA = [i for i, x in enumerate(ind1) if x == 'A']
    posB = [i for i, x in enumerate(ind1) if x == 'B']
    # Obtenemos las posiciones de 0 de ambos padres
    pos0_1 = [i for i, x in enumerate(ind1) if x == 0]
    pos0_2 = [i for i, x in enumerate(ind2) if x == 0]
    # Seleccionamos un día al azar
    dia = random.randint(0, len(posB)-1)
    # Seleccionamos un quirófano al azar del día seleccionado
    quirofano = random.randint(posA[dia], posB[dia]-1)
    # Seleccionamos un punto de corte al azar del día y quirófano seleccionados
    puntoCorte = random.randint(pos0_1[quirofano], pos0_2[quirofano])
    # Creamos los hijos
    hijo1 = ind1[:posA[quirofano]] + ind2[posA[quirofano]:posB[dia]] + ind1[posB[dia]:]
    hijo2 = ind2[:posA[quirofano]] + ind1[posA[quirofano]:posB[dia]] + ind2[posB[dia]:]
    # Intercambiamos los genes de los puntos de corte
    hijo1[pos0_1[puntoCorte]], hijo2[pos0_2[puntoCorte]] = hijo2[pos0_2[puntoCorte]], hijo1[pos0_1[puntoCorte]]
    return hijo1, hijo2
def mutacion(individual):
    # Intercambia dos genes de posición
    pos1 = random.randint(0, len(individual)-1)
    pos2 = random.randint(0, len(individual)-1)
    individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
    return individual

In [7]:
toolbox.register("evaluate", evaluar)
toolbox.register("mate", cruce)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [8]:
# Definimos los parámetros del algoritmo
# Tamaño de la población
tPoblacion = 100
# Probabilidad de cruce
pCruce = 0.8
# Probabilidad de mutación
pMutacion = 0.2
# Número de generaciones
nGeneraciones = 1000
# Creamos la población inicial
poblacion = toolbox.poblacion(n=tPoblacion)
# Evaluamos la población inicial
fitnesses = list(map(toolbox.evaluate, poblacion))
for ind, fit in zip(poblacion, fitnesses):
    ind.fitness.values = fit
# Guardamos el mejor individuo
mejor = tools.selBest(poblacion, 1)[0]
# Guardamos el mejor fitness
mejorFitness = mejor.fitness.values[0]
# Guardamos el fitness medio
fitnessMedio = 0
# Guardamos el fitness del peor individuo
peorFitness = 0
# Guardamos el fitness del mejor individuo de cada generación
mejoresFitness = []
# Guardamos el fitness medio de cada generación
fitnessesMedios = []
# Guardamos el fitness del peor individuo de cada generación
peoresFitness = []
# Guardamos el número de generaciones sin mejora
nGeneracionesSinMejora = 0
# Ejecutamos el algoritmo
for i in range(nGeneraciones):
    # Seleccionamos la siguiente generación
    offspring = toolbox.select(poblacion, len(poblacion))
    # Clonamos los individuos seleccionados
    offspring = list(map(toolbox.clone, offspring))
    # Aplicamos el cruce
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < pCruce:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    # Aplicamos la mutación
    for mutant in offspring:
        if random.random() < pMutacion:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    # Evaluamos los individuos con fitness inválido
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    # Reemplazamos la población por la nueva generación
    poblacion[:] = offspring
    # Guardamos el mejor individuo
    mejor = tools.selBest(poblacion, 1)[0]
    # Guardamos el mejor fitness
    mejorFitness = max(mejorFitness, mejor.fitness.values[0])
    # Guardamos el fitness medio
    fitnessMedio = sum([ind.fitness.values[0] for ind in poblacion])/len(poblacion)
    # Guardamos el fitness del peor individuo
    peorFitness = min([ind.fitness.values[0] for ind in poblacion])
    # Guardamos el fitness del mejor individuo de cada generación
    mejoresFitness.append(mejorFitness)
    # Guardamos el fitness medio de cada generación
    fitnessesMedios.append(fitnessMedio)
    # Guardamos el fitness del peor individuo de cada generación
    peoresFitness.append(peorFitness)
    # Si el mejor individuo no ha mejorado, incrementamos el contador
    if len(mejoresFitness)>=2 and mejorFitness == mejoresFitness[-2]:
        nGeneracionesSinMejora += 1
    else:
        nGeneracionesSinMejora = 0
    # Si no ha habido mejora en las últimas 50 generaciones, paramos
    if nGeneracionesSinMejora == 50:
        break

# Mostramos los resultados
print("Mejor individuo: ", mejor)
print("Mejor fitness: ", mejorFitness)
print("Fitness medio: ", fitnessMedio)
print("Peor fitness: ", peorFitness)
# Mostramos la gráfica
plt.plot(mejoresFitness, label="Mejor")
plt.plot(fitnessesMedios, label="Medio")
plt.plot(peoresFitness, label="Peor")
plt.legend()
plt.show()

IndexError: list index out of range